In [1]:
import importlib.util
import importlib.abc
import sys
import os


class CustomLoader(importlib.abc.Loader):
    def __init__(self, fullname, path):
        self.fullname = fullname
        self.path = path

    def create_module(self, spec):
        return None  # Use default module creation

    def exec_module(self, module):
        with open(self.path, 'r', encoding='utf-8') as f:
            source = f.read()
        exec(source, module.__dict__)


class CustomFinder(importlib.abc.MetaPathFinder):
    def __init__(self, base_dir):
        self.base_dir = os.path.abspath(base_dir)

    def find_spec(self, fullname, path, target=None):
        parts = fullname.split('.')
        module_path = os.path.join(self.base_dir, *parts)

        # Check for package (__init__.py)
        init_path = os.path.join(module_path, '__init__.py')
        if os.path.isfile(init_path):
            loader = CustomLoader(fullname, init_path)
            return importlib.util.spec_from_file_location(
                fullname, init_path, loader=loader, submodule_search_locations=[module_path]
            )

        # Check for module (.py file)
        file_path = module_path + '.py'
        if os.path.isfile(file_path):
            loader = CustomLoader(fullname, file_path)
            return importlib.util.spec_from_file_location(fullname, file_path, loader=loader)

        return None


def install_custom_importer(directory):
    sys.meta_path.insert(0, CustomFinder(directory))


class CustomImporter:
    def __init__(self, base_dir):
        self.finder = CustomFinder(base_dir)

    def __enter__(self):
        sys.meta_path.insert(0, self.finder)

    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.finder in sys.meta_path:
            sys.meta_path.remove(self.finder)


In [4]:
install_custom_importer(r'C:\Users\suraj\projects\async_risk')